In [1]:

import sys, os
cdir = %pwd
sys.path.append(os.path.join(cdir, '..', 'python'))


In [6]:
from hue_show import *

In [7]:
!curl https://www.meethue.com/api/nupnp

[{"id":"001788fffe0aa167","internalipaddress":"192.168.1.113"}]

In [8]:
from time import sleep
from phue import Bridge
import random
from numpy import zeros, ones
import numpy as np
import matplotlib.pyplot as plt
from tqdm import *
import requests as req
import pandas as pd
from rgb_cie import Converter

In [75]:
bridges = req.get('https://www.meethue.com/api/nupnp').json()

In [76]:
bridges

[{'id': '001788fffe273a83', 'internalipaddress': '192.168.1.21'},
 {'id': '001788fffe0aa167', 'internalipaddress': '192.168.1.113'}]

The different bridges have the following addresse:

In [88]:
import yaml

In [89]:
with open('bridges.yml', 'r') as f:
    bridges = yaml.load(f.read())

In [91]:
get_bridge = lambda name: Bridge('{IP}:{port}'.format(**bridges[name]), bridges[name]['user'])

In [94]:
b = get_bridge('home1')

In [95]:
b.lights

dict_values([<phue.Light object "AJ" at 0x7ff0b5000198>, <phue.Light object "P10" at 0x7ff0b50005c0>, <phue.Light object "P9" at 0x7ff0b50005f8>, <phue.Light object "P11" at 0x7ff0b5000588>, <phue.Light object "P12" at 0x7ff0b50004e0>, <phue.Light object "P13" at 0x7ff0b5000748>, <phue.Light object "P8" at 0x7ff0b5000550>, <phue.Light object "P18" at 0x7ff0b50006a0>, <phue.Light object "P5" at 0x7ff0b478a278>, <phue.Light object "P15" at 0x7ff0b4ff08d0>, <phue.Light object "P14" at 0x7ff0b5000390>, <phue.Light object "P3" at 0x7ff0b5000240>, <phue.Light object "P7" at 0x7ff0b5000710>, <phue.Light object "P17" at 0x7ff0b478a198>, <phue.Light object "P6" at 0x7ff0b5000128>, <phue.Light object "P4" at 0x7ff0b478a2e8>, <phue.Light object "P2" at 0x7ff0b478a160>, <phue.Light object "P1" at 0x7ff0b478a0f0>, <phue.Light object "P16" at 0x7ff0b478a240>, <phue.Light object "P19" at 0x7ff0b478a208>, <phue.Light object "Hue color lamp 1" at 0x7ff0b5000278>, <phue.Light object "Hue color lamp 2" a

In [96]:
get('home1', '')

{'config': {'UTC': '2016-11-20T15:03:48',
  'apiversion': '1.15.0',
  'backup': {'errorcode': 0, 'status': 'idle'},
  'bridgeid': '001788FFFE273A83',
  'dhcp': True,
  'factorynew': False,
  'gateway': '192.168.1.1',
  'ipaddress': '192.168.1.21',
  'linkbutton': False,
  'localtime': '2016-11-20T16:03:48',
  'mac': '00:17:88:27:3a:83',
  'modelid': 'BSB002',
  'name': 'Philips hue',
  'netmask': '255.255.255.0',
  'portalconnection': 'connected',
  'portalservices': True,
  'portalstate': {'communication': 'connected',
   'incoming': True,
   'outgoing': True,
   'signedon': True},
  'proxyaddress': 'none',
  'proxyport': 0,
  'replacesbridgeid': '001788FFFE1C6E2C',
  'swupdate': {'checkforupdate': False,
   'devicetypes': {'bridge': False, 'lights': [], 'sensors': []},
   'notify': False,
   'text': 'ZGB_100B_0100 - 5.23.1.13452',
   'updatestate': 2,
   'url': ''},
  'swversion': '01035934',
  'timezone': 'Europe/Copenhagen',
  'whitelist': {'1461b5d694b73cd5655ee307b6f3a0c': {'crea

In [86]:
bri = []
for br in bridges:
    b = Bridge(br['internalipaddress'])
    b.connect
    bri.append(b)

ConnectionRefusedError: [Errno 111] Connection refused

In [87]:
b = Bridge(bridges[0]['internalipaddress'])

ConnectionRefusedError: [Errno 111] Connection refused

In [88]:
b = Bridge(bridges[1]['internalipaddress'])

In [41]:
ln = 1
speed = 20 # m/s
distance = 7 #m between each lamp
time = 7 / (speed * 1E3 / (60*60))

def light(name, r_,g_,b_):
    l = b.lights_by_name[name]
    rgbl(l, r_,g_,b_, brightness=254)

normal(b)    
for n in range(9):
    try:
        name = "P%d"%((ln+n)%(19)+1)
        l = b.lights_by_name[name]
        rgbl(l, 1.0,0.0,0.0, brightness=254)
        sleep(time)
        rgbl(l, 1.,1.,1., brightness=254)
        sleep(0.1)
    except Exception as e:
        print(e)


KeyError: 'P1'

In [44]:
b.lights_by_name

{}

### Search for lights

In [45]:
import json
import time

In [46]:
b1 = b1[0]

IndexError: list index out of range

In [49]:
IP = b.ip

In [45]:
string_format = lambda i: 'http://{IP}:{port}/api/{user}/'.format(**bridges[i])
get = lambda i, str_:req.get(string_format(i)+str_).json()
post = lambda i, str_, data:req.post(string_format(i)+str_, json.dumps(data)).json()
put = lambda i, str_,  data:req.put(string_format(i)+str_, json.dumps(data)).json()

In [60]:
get('hue_in', '')

{'config': {'UTC': '2016-11-20T14:48:33',
  'apiversion': '1.15.0',
  'backup': {'errorcode': 0, 'status': 'idle'},
  'bridgeid': '001788FFFE1C474C',
  'dhcp': True,
  'factorynew': False,
  'gateway': '192.168.0.1',
  'ipaddress': '192.168.0.199',
  'linkbutton': False,
  'localtime': '2016-11-20T15:48:33',
  'mac': '00:17:88:1c:47:4c',
  'modelid': 'BSB001',
  'name': 'HUE inde',
  'netmask': '255.255.255.0',
  'portalconnection': 'connected',
  'portalservices': True,
  'portalstate': {'communication': 'disconnected',
   'incoming': True,
   'outgoing': True,
   'signedon': True},
  'proxyaddress': 'none',
  'proxyport': 0,
  'replacesbridgeid': None,
  'swupdate': {'checkforupdate': False,
   'devicetypes': {'bridge': False, 'lights': [], 'sensors': []},
   'notify': True,
   'text': '',
   'updatestate': 1,
   'url': ''},
  'swversion': '01036562',
  'timezone': 'Europe/Copenhagen',
  'whitelist': {'0000000065a912e14d68ef084d68ef08': {'create date': '2015-11-07T20:50:09',
    'las

Search for new lights

In [52]:
post(0,'lights',{})
post(1,'lights',{})

IndexError: list index out of range

In [53]:
get(0,'lights/new')
get(1,'lights/new')

IndexError: list index out of range

In [59]:
print('now there are %d lights'%(len(get(0,'lights'))))

IndexError: list index out of range

In [58]:
lamps = {}
for i in range(len(bri)):
    print('Bridge-%d'%(i))
    for k,v in get(i,'lights').items():
        #if 'Hue color' in v['name']:
        print(v['name'], v['state']['reachable'], v['uniqueid'])
        idl = v['uniqueid']
        if idl in lamps:
            lamps[idl][i] = v
        else:
            lamps[idl] = {i:v}
    time.sleep(2)

In [57]:
for k,v in lamps.items():
    if len(v)>1:
        print('0:',   v[0]['state']['reachable'], v[0]['name'], 
              '\n1:', v[1]['state']['reachable'], v[1]['name'], 
              '\n')

In [73]:
lamps2 = {}
for k, v in lamps.items():
    if 0 in v:
        if 'P' in v[0]['name']:
                lamps2[v[0]['name']] = v
    if 1 in v:
        if 'P' in v[1]['name']:
            lamps2[v[1]['name']] = v
        

In [60]:
lamps2['P1']

NameError: name 'lamps2' is not defined

In [62]:
len(all_lights[0:10])

10

In [63]:
import json

In [64]:
import time

In [65]:
for i in range(len(bri)):
    post(i,'lights', {"deviceid":all_lights[0:10]})
    time.sleep(2)
    post(i,'lights', {"deviceid":all_lights[10:]})
    time.sleep(2)
    

In [66]:
bri

[]

### PARTY MODE

In [66]:
for i in range(len(bri)):
    put(i, 'groups/0/action', 
        {"on": True,
         "hue": 2000,
        })
    time.sleep(1)

In [ ]:
class Controler(object):
    def __init__(self):
        self.targets = {}
        

In [1]:
b = Bridge(bridges[1]['internalipaddress'])

NameError: name 'Bridge' is not defined

In [ ]:
x